# Install Semantic Link

In [ ]:
!pip install semantic-link -q

# Initialize Lakehouse Names

In [ ]:
raw_lakehouse = "lh_bronze"
clean_lakehouse = "lh_gold"

# Set Git Links

In [ ]:
REPO_NAME = "Fabric-Cricsheet"
MAIN_NAME = f"/{REPO_NAME}-main"
GITHUB_REPO_ZIP = f"https://github.com/akhilannan/{REPO_NAME}/archive/refs/heads/main.zip"
SEMANTIC_MODEL = "Cricsheet Model"
REPORT_NAME = "Cricsheet Analysis"
BIM_PATH = f"{MAIN_NAME}/Semantic Model/{SEMANTIC_MODEL}.bim"
REPORT_PATH = f"{MAIN_NAME}/Power BI Report/{REPORT_NAME}.Report/report.json"
NOTEBOOK_PATH = f"{MAIN_NAME}/Notebooks"
UTILITY_PATH = f"{MAIN_NAME}/Utility Functions"
FABRIC_UTILS_FILE_NAME = "fabric_utils.py"

# Create Mount Points

In [ ]:
lakehouses = {raw_lakehouse: None, clean_lakehouse: None}
for lh in lakehouses:
    try:
        # Attempt to create the lakehouse and update the dictionary if successful
        lakehouse_details = mssparkutils.lakehouse.create(lh)
    except:
        # If creation fails, get the existing lakehouse and update the dictionary
        lakehouse_details = mssparkutils.lakehouse.get(lh)
    workspace_id = lakehouse_details.workspaceId
    lakehouse_id = lakehouse_details.id
    abfss_lakehouse_path = f"abfss://{workspace_id}@onelake.dfs.fabric.microsoft.com/{lakehouse_id}"
    mount_point = f"/lakehouse/{lh}"
    mssparkutils.fs.mount(abfss_lakehouse_path, mount_point)
    lakehouses[lh] = mssparkutils.fs.getMountPath(mount_point)


# Initialize Variables

In [ ]:
import os
files_path = "/Files"
lh_files_raw_path = lakehouses[raw_lakehouse] + files_path
lh_git_raw_path = lh_files_raw_path + "/git"
lh_bim_path = lh_git_raw_path + BIM_PATH
lh_report_path = lh_git_raw_path + REPORT_PATH
lh_git_notebook_path = lh_git_raw_path + NOTEBOOK_PATH
git_utils_path = lh_files_raw_path + "/git" + UTILITY_PATH + "/" + FABRIC_UTILS_FILE_NAME
package_path = os.path.join(lh_files_raw_path, "packages")
fabric_path = os.path.join(package_path, "fabric_python_functions")
fabric_utils_path = os.path.join(fabric_path, "fabric_utils")

# Download Git Repo contents as Zip and Unzip it

In [ ]:
import requests
import zipfile
import io
import shutil

shutil.rmtree(lh_git_raw_path, ignore_errors=True)
r = requests.get(GITHUB_REPO_ZIP)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(lh_git_raw_path)

# Add and Import Fabric Utilities

In [ ]:
sc.addPyFile(git_utils_path)
import fabric_utils as U

# Create Wheel Package

In [ ]:
import subprocess
import sys

# Remove the existing package directory and create a new one
shutil.rmtree(package_path, ignore_errors=True)
os.makedirs(fabric_utils_path, exist_ok=True)

# Write the Python code to the functions.py file
python_file_path = os.path.join(fabric_utils_path, 'functions.py')
shutil.copyfile(git_utils_path, python_file_path)

# Extract the function names to a list
function_names =[]
with open(python_file_path, 'r') as file:
    for line in file:
        # Check if the line contains a function definition
        if line.startswith('def '):
            # Extract the function name
            function_name = line.split('def ')[1].split('(')[0]
            function_names.append(function_name)

# Write the __init__.py content
init_content = "\n".join(f"from .functions import {name}" for name in function_names)
init_file_path = os.path.join(fabric_utils_path, '__init__.py')
with open(init_file_path, 'w') as init_file:
    init_file.write(init_content)

# Write the setup.py content
setup_content = """
from setuptools import setup, find_packages

setup(
name='fabric_utils',
version='0.1',
packages=find_packages(),
install_requires=['semantic-link'],
)
"""
setup_file_path = os.path.join(fabric_path, 'setup.py')
with open(setup_file_path, 'w') as setup_file:
    setup_file.write(setup_content)

# Build the wheel file
os.chdir(fabric_path)
subprocess.check_call([sys.executable, 'setup.py', 'sdist', 'bdist_wheel'])

# Move the wheel file outside the 'fabric_utils' folder
wheel_files = [f for f in os.listdir('dist') if f.endswith('.whl')]
for file in wheel_files:
    shutil.move(f'dist/{file}', f'../{file}')

# Clean up the package directory
shutil.rmtree(fabric_path)

# Print a success message
print("The wheel file has been successfully created and moved outside the 'fabric_utils' folder.")


# Copy Wheel file to Clean Lakehouse

In [ ]:
clean_package_path = package_path.replace(raw_lakehouse, clean_lakehouse)
if os.path.exists(clean_package_path):
        shutil.rmtree(clean_package_path)
shutil.copytree(package_path, clean_package_path)

# Deploy Notebook

In [ ]:
import json

# Define the code replacements using formatted strings
code_replacements = {
    'RAW_LAKEHOUSE = "lh_bronze"': f'RAW_LAKEHOUSE = "{raw_lakehouse}"',
    'CLEAN_LAKEHOUSE = "lh_gold"': f'CLEAN_LAKEHOUSE = "{clean_lakehouse}"'
}

# Define the mapping of notebook names to their respective default lakehouse
notebook_lakehouse = {
    'Cricsheet Orchestrator': raw_lakehouse,
    'Cricsheet Initialize': raw_lakehouse,
    'Cricsheet Ingest Data': raw_lakehouse,
    'Cricsheet Build Facts and Dimensions': clean_lakehouse,
    'Cricsheet Model Refresh': clean_lakehouse,
    'Cricsheet Optimize and Vacuum': raw_lakehouse
}

# Iterate over the notebook_lakehouse dictionary and process each notebook
for file_name, lakehouse_type in notebook_lakehouse.items():
    full_file_name = file_name + ".ipynb"
    file_path = os.path.join(lh_git_notebook_path , full_file_name)
    with open(file_path, 'r') as file:
        notebook_json = json.load(file)
    U.create_or_replace_notebook_from_ipynb(file_name, notebook_json, lakehouse_type, code_replacements)

# Deploy Semantic Model

In [ ]:
with open(lh_bim_path, 'r') as file:
        bim_json = json.load(file)
U.execute_with_retries(U.create_or_replace_semantic_model_from_bim,
                     dataset_name=SEMANTIC_MODEL,
                     bim_file_json=bim_json)

# Update Semantic Model Connection

In [ ]:
U.execute_with_retries(U.update_model_expression,
                     dataset_name=SEMANTIC_MODEL,
                     lakehouse_name=clean_lakehouse)

# Deploy Report and Repoint to the Semantic Model

In [ ]:
with open(lh_report_path, 'r') as file:
        report_json = json.load(file)
U.execute_with_retries(U.create_or_replace_report_from_reportjson,
                     report_name=REPORT_NAME,
                     dataset_name=SEMANTIC_MODEL,
                     report_json=report_json)